In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session with Iceberg configurations
spark = SparkSession.builder \
  .appName("IcebergLocalDevelopment") \
  .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1') \
  .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
  .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
  .config("spark.sql.catalog.local.type", "hadoop") \
  .config("spark.sql.catalog.local.warehouse", "spark-warehouse/iceberg") \
  .getOrCreate()

25/10/01 15:15:59 WARN Utils: Your hostname, joaovfe resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/01 15:15:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/root/projects/engenharia_de_dados/spark-delta-iceberg/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e6073a61-44df-4b88-aaf4-9e96d43f036f;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 in central
:: resolution report :: resolve 144ms :: artifacts dl 4ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-subm

In [2]:
spark

In [6]:
df = spark.read.option("header", True).csv("../../data/listings.csv")

df = df.selectExpr(
    "id as listing_id",
    "host_id",
    "host_name",
    "neighbourhood as city",
    "room_type as property_type",
    "cast(price as double) as price"
)

df.show()


+----------+-------+--------------------+---------------+---------------+------+
|listing_id|host_id|           host_name|           city|  property_type| price|
+----------+-------+--------------------+---------------+---------------+------+
|     17878|  68997|            Matthias|     Copacabana|Entire home/apt| 254.0|
|     25026| 102840|             Viviane|     Copacabana|Entire home/apt| 252.0|
|     35764| 153691|Patricia Miranda ...|     Copacabana|Entire home/apt| 190.0|
|     48305|  70933|             Goitaca|        Ipanema|Entire home/apt|2239.0|
|     48901| 222884|              Marcio|     Copacabana|Entire home/apt| 743.0|
|     49179| 224192|               David|     Copacabana|Entire home/apt| 189.0|
|     50759| 233554|              Felipe|Barra da Tijuca|Entire home/apt|2800.0|
|     51703| 238091|               Dalia|     Copacabana|Entire home/apt| 201.0|
|     53533| 249439|      Sherri & Andre|            Joá|Entire home/apt|1467.0|
|     64795|  93005|        

In [7]:
spark.sql("""
CREATE TABLE local.db_listings (
    listing_id STRING,
    host_id STRING,
    host_name STRING,
    city STRING,
    property_type STRING,
    price DOUBLE
) USING iceberg
""")


DataFrame[]

In [ ]:
spark.sql("SELECT * FROM local.db_listings").show()

In [8]:
df.createOrReplaceTempView("temp_listings")

spark.sql("""
INSERT INTO local.db_listings
SELECT * FROM temp_listings
""")

DataFrame[]

In [9]:
spark.sql("SELECT * FROM local.db_listings").show()

+----------+-------+--------------------+---------------+---------------+------+
|listing_id|host_id|           host_name|           city|  property_type| price|
+----------+-------+--------------------+---------------+---------------+------+
|     17878|  68997|            Matthias|     Copacabana|Entire home/apt| 254.0|
|     25026| 102840|             Viviane|     Copacabana|Entire home/apt| 252.0|
|     35764| 153691|Patricia Miranda ...|     Copacabana|Entire home/apt| 190.0|
|     48305|  70933|             Goitaca|        Ipanema|Entire home/apt|2239.0|
|     48901| 222884|              Marcio|     Copacabana|Entire home/apt| 743.0|
|     49179| 224192|               David|     Copacabana|Entire home/apt| 189.0|
|     50759| 233554|              Felipe|Barra da Tijuca|Entire home/apt|2800.0|
|     51703| 238091|               Dalia|     Copacabana|Entire home/apt| 201.0|
|     53533| 249439|      Sherri & Andre|            Joá|Entire home/apt|1467.0|
|     64795|  93005|        

In [10]:
spark.sql("SELECT * FROM local.db_listings WHERE listing_id = '35764'").show()

+----------+-------+--------------------+----------+---------------+-----+
|listing_id|host_id|           host_name|      city|  property_type|price|
+----------+-------+--------------------+----------+---------------+-----+
|     35764| 153691|Patricia Miranda ...|Copacabana|Entire home/apt|190.0|
+----------+-------+--------------------+----------+---------------+-----+



In [11]:
spark.sql("""
DELETE FROM local.db_listings
WHERE listing_id = '35764'
""")

DataFrame[]

In [12]:
spark.sql("SELECT * FROM local.db_listings WHERE listing_id = '35764'").show()

+----------+-------+---------+----+-------------+-----+
|listing_id|host_id|host_name|city|property_type|price|
+----------+-------+---------+----+-------------+-----+
+----------+-------+---------+----+-------------+-----+



In [13]:
spark.sql("SELECT * FROM local.db_listings WHERE listing_id = '17878'").show()

+----------+-------+---------+----------+---------------+-----+
|listing_id|host_id|host_name|      city|  property_type|price|
+----------+-------+---------+----------+---------------+-----+
|     17878|  68997| Matthias|Copacabana|Entire home/apt|254.0|
+----------+-------+---------+----------+---------------+-----+



In [14]:
spark.sql("""
UPDATE local.db_listings
SET price = 500
WHERE listing_id = '17878'
""")

DataFrame[]

In [15]:
spark.sql("SELECT * FROM local.db_listings WHERE listing_id = '17878'").show()

+----------+-------+---------+----------+---------------+-----+
|listing_id|host_id|host_name|      city|  property_type|price|
+----------+-------+---------+----------+---------------+-----+
|     17878|  68997| Matthias|Copacabana|Entire home/apt|500.0|
+----------+-------+---------+----------+---------------+-----+



In [16]:
spark.sql("DROP TABLE local.db_listings")

DataFrame[]